In [1]:
# Required imports for the dungeon adventure game
from IPython.display import clear_output
import random
from collections import deque

In [8]:
# Quest management classes
class Quest:
    def __init__(self, name, desc, done=False):
        self.name, self.desc, self.done, self.next = name, desc, done, None

class QuestLog:
    def __init__(self):
        self.head = None
    
    def add(self, name, desc):
        new_q = Quest(name, desc)
        new_q.next, self.head = self.head, new_q
    
    def complete(self, name):
        curr = self.head
        while curr:
            if curr.name == name:
                curr.done = True
                return True
            curr = curr.next
        return False
    
    def display(self):
        if not self.head: return "📋 No quests"
        result, curr = "📋 Quest Log:\n", self.head
        while curr:
            result += f"{'✅' if curr.done else '📝'} {curr.name}: {curr.desc}\n"
            curr = curr.next
        return result.strip()

In [9]:
class Item:
    def __init__(self, name, emoji, heal=0, attack=0):
        self.name, self.emoji, self.heal, self.attack = name, emoji, heal, attack
    
    def use(self, player):
        if self.heal:
            old_hp = player.hp
            player.hp = min(player.hp + self.heal, 100)
            return f"Healed {player.hp - old_hp} HP!"
        return "Item cannot be used."

class Entity:
    def __init__(self, x, y, hp=100):
        self.x, self.y, self.hp, self.max_hp = x, y, hp, hp

class Player(Entity):
    def __init__(self, x=1, y=1):
        super().__init__(x, y, 100)
        self.inv, self.weapon, self.cooldown = [], None, 0
    
    def move(self, direction, game_map, monsters):
        dirs = {"north":(0,-1), "south":(0,1), "west":(-1,0), "east":(1,0)}
        if direction not in dirs: return False
        
        dx, dy = dirs[direction]
        nx, ny = self.x + dx, self.y + dy
        
        if not (0 <= nx < game_map.w and 0 <= ny < game_map.h): return False
        if game_map.grid[ny][nx] == '#': return False
        if any(m.x == nx and m.y == ny and m.hp > 0 for m in monsters): return False
        
        self.x, self.y = nx, ny
        if self.cooldown > 0: self.cooldown -= 1
        return True
    
    def can_attack(self): return self.cooldown == 0
    def attack(self): 
        if self.can_attack(): 
            self.cooldown = 2
            return True
        return False

class Monster(Entity):
    def __init__(self, x, y, name="👹", hp=10):
        super().__init__(x, y, hp)
        self.name, self.move_chance, self.rest = name, 0.3, 0
    
    def find_path(self, game_map, target):
        if (self.x, self.y) == target: return []
        queue, visited = deque([((self.x, self.y), [])]), {(self.x, self.y)}
        
        while queue:
            (x, y), path = queue.popleft()
            for dx, dy in [(0,-1), (0,1), (-1,0), (1,0)]:
                nx, ny = x + dx, y + dy
                if (0 <= nx < game_map.w and 0 <= ny < game_map.h and 
                    game_map.grid[ny][nx] != '#' and (nx, ny) not in visited):
                    new_path = path + [(nx, ny)]
                    if (nx, ny) == target: return new_path
                    queue.append(((nx, ny), new_path))
                    visited.add((nx, ny))
        return []
    
    def move_to_player(self, game_map, player_pos, monsters):
        if self.hp <= 0 or self.rest > 0: return
        if abs(self.x - player_pos[0]) + abs(self.y - player_pos[1]) == 1: return
        
        occupied = {(m.x, m.y) for m in monsters if m.hp > 0 and m != self} | {player_pos}
        
        if random.random() <= self.move_chance:
            path = self.find_path(game_map, player_pos)
            if path and path[0] not in occupied:
                self.x, self.y = path[0]
                return
        
        # Random move
        dirs = [(0,-1), (0,1), (-1,0), (1,0)]
        random.shuffle(dirs)
        for dx, dy in dirs:
            nx, ny = self.x + dx, self.y + dy
            if (0 <= nx < game_map.w and 0 <= ny < game_map.h and 
                game_map.grid[ny][nx] != '#' and (nx, ny) not in occupied):
                self.x, self.y = nx, ny
                break


In [10]:

class AdventureMap:
    def __init__(self, diff="easy"):
        self.diff, self.items = diff, {}
        inside = self.get_map()
        self.grid = [["#"] * (len(inside[0]) + 2)]
        for row in inside: self.grid.append(["#"] + row + ["#"])
        self.grid.append(["#"] * (len(inside[0]) + 2))
        self.h, self.w = len(self.grid), len(self.grid[0])
    
    def get_map(self):
        sword, potion = Item("Sword", "⚔️", 0, 40), Item("Health Potion", "🧪", 30)
        
        maps = {
            "easy": ([
                list("     #   #     "),
                list("  #  # # #     "),
                list("  #        #   "),
                list(" #  ###  #     "),
                list("      #        "),
                list("  #   #   #    "),
                list("  #        #   "),
                list("   # ### #     "),
                list("      #        "),
            ], {(1,5): sword, (3,7): potion}),  # Fixed coordinates: (x+1, y+1) to account for border
            "medium": ([
                list("  #   #   #   #  "),
                list(" # # #   # # #   "),
                list("    #       #    "),
                list(" #   ### ###   # "),
                list("     #   #       "),
                list(" #   #   #   #   "),
                list("   #       #     "),
                list(" # ###   ### #   "),
                list("       #         "),
            ], {(9,5): sword, (4,8): potion}),  # Fixed coordinates
            "hard": ([
                list("  # #   #   # #   "),
                list(" #   # # # #   #  "),
                list("   #       #      "),
                list(" #  ### ### ###  #"),
                list("     #   #   #    "),
                list("  #   #     #   # "),
                list("    #       #     "),
                list(" # ### # # # ### #"),
                list("      #       #   "),
            ], {(11,5): sword, (6,9): potion})  # Fixed coordinates
        }
        
        inside, items = maps[self.diff]
        self.items = items
        return inside
    
    def display(self, player, monsters):
        display_grid = [row[:] for row in self.grid]
        
        for pos, item in self.items.items():
            if 0 <= pos[0] < self.w and 0 <= pos[1] < self.h:
                display_grid[pos[1]][pos[0]] = item.emoji
        
        display_grid[player.y][player.x] = "P"
        for m in monsters:
            if m.hp > 0: display_grid[m.y][m.x] = m.name
        
        emoji_map = {"#": "🪨", " ": "⬛", "P": "🧙"}
        
        for y in range(self.h):
            row = ""
            for x in range(self.w):
                cell = display_grid[y][x]
                row += emoji_map.get(cell, cell)
            print(row)



In [11]:
class Game:
    def __init__(self, diff="easy"):
        self.map = AdventureMap(diff)
        self.player = Player()
        self.monsters = self.create_monsters(diff)
        self.msg = ""
        self.fists = Item("Fists", "👊", 0, 5)
        self.quests = QuestLog()
        
        # Simple undo - just store essential values
        self.undo_history = []
        self.max_undo = 5
        
        self.quests.add("Defeat All Monsters", "Clear the dungeon of all enemies")
        self.player.inv.append(Item("Health Potion", "🧪", 30))
        self.save_simple_state("Game started")
    
    def create_monsters(self, diff):
        configs = {
            "easy": [(6,4,"👾",15), (11,4,"👾",15), (8,8,"👹",25)],  # Fixed coordinates
            "medium": [(6,4,"👾",20), (10,7,"👺",35), (14,9,"👹",50)],  # Fixed coordinates
            "hard": [(6,4,"👾",25), (10,7,"👺",40), (13,6,"👹",60), (15,9,"💀",80)]  # Fixed coordinates
        }
        
        monsters = []
        for x, y, name, hp in configs[diff]:
            m = Monster(x, y, name, hp)
            if diff == "hard":
                m.move_chance = [0.8, 0.6, 0.6, 0.5][len(monsters)]
            monsters.append(m)
        return monsters
    
    def save_simple_state(self, action=""):
        # Just save the numbers we need - no fancy copying
        monster_data = []
        for m in self.monsters:
            monster_data.append((m.x, m.y, m.hp, m.rest))
        
        item_positions = list(self.map.items.keys())
        
        inv_names = []
        for item in self.player.inv:
            inv_names.append(item.name)
        
        weapon_name = self.player.weapon.name if self.player.weapon else None
        
        state = {
            'action': action,
            'px': self.player.x,
            'py': self.player.y,
            'php': self.player.hp,
            'pcd': self.player.cooldown,
            'monsters': monster_data,
            'items': item_positions,
            'inv': inv_names,
            'weapon': weapon_name
        }
        
        self.undo_history.append(state)
        if len(self.undo_history) > self.max_undo:
            self.undo_history.pop(0)
    
    def undo(self):
        if len(self.undo_history) < 2:
            self.msg = "❌ Nothing to undo!"
            return False
        
        self.undo_history.pop()  # Remove current state
        prev = self.undo_history[-1]  # Get previous state
        
        # Restore player
        self.player.x = prev['px']
        self.player.y = prev['py']
        self.player.hp = prev['php']
        self.player.cooldown = prev['pcd']
        
        # Restore monsters
        for i, (x, y, hp, rest) in enumerate(prev['monsters']):
            if i < len(self.monsters):
                self.monsters[i].x = x
                self.monsters[i].y = y
                self.monsters[i].hp = hp
                self.monsters[i].rest = rest
        
        # Restore items on map - Fixed the coordinate matching
        self.map.items.clear()
        for pos in prev['items']:
            # Check for sword positions (updated coordinates)
            if pos in [(1,5), (9,5), (11,5)]:  # Updated sword positions
                self.map.items[pos] = Item("Sword", "⚔️", 0, 40)
            # Check for potion positions (updated coordinates)  
            elif pos in [(3,7), (4,8), (6,9)]:  # Updated potion positions
                self.map.items[pos] = Item("Health Potion", "🧪", 30)
        
        # Restore inventory
        self.player.inv.clear()
        for item_name in prev['inv']:
            if item_name == "Health Potion":
                self.player.inv.append(Item("Health Potion", "🧪", 30))
            elif item_name == "Sword":
                self.player.inv.append(Item("Sword", "⚔️", 0, 40))
        
        # Restore weapon
        self.player.weapon = None
        if prev['weapon']:
            for item in self.player.inv:
                if item.name == prev['weapon']:
                    self.player.weapon = item
                    break
        
        self.msg = f"⏪ Undid: {prev['action']}"
        return True
    
    def get_adjacent_monster(self):
        for dx, dy in [(0,-1), (0,1), (-1,0), (1,0)]:
            for m in self.monsters:
                if m.x == self.player.x + dx and m.y == self.player.y + dy and m.hp > 0:
                    return m
        return None
    
    def move_monsters(self):
        for m in self.monsters:
            if m.hp > 0: m.move_to_player(self.map, (self.player.x, self.player.y), self.monsters)
    
    def monster_attack(self):
        adj = self.get_adjacent_monster()
        if adj and adj.rest <= 0:
            dmg = random.randint(5, 15)
            self.player.hp -= dmg
            self.msg += f"💥 {adj.name} hits you for {dmg} damage!\n"
            adj.rest = 2
        
        for m in self.monsters:
            if m.rest > 0: m.rest -= 1
    
    def display_monsters(self):
        alive = [m for m in self.monsters if m.hp > 0]
        if not alive: return
        print("\n🧿 Monsters:")
        for m in alive:
            dist = abs(m.x - self.player.x) + abs(m.y - self.player.y)
            hp_pct = m.hp / m.max_hp
            bar = "❤️" * int(hp_pct * 10) + "🖤" * (10 - int(hp_pct * 10))
            print(f"{m.name} HP: {bar} ({m.hp}/{m.max_hp}) | 📏 {dist}")
    
    def draw(self):
        clear_output(wait=True)
        self.map.display(self.player, self.monsters)
        
        weapon = self.player.weapon or self.fists
        cd = f" ⏳{self.player.cooldown}" if self.player.cooldown > 0 else ""
        print(f"\n⚔️ Weapon: {weapon.emoji} {weapon.name} (⚔️ {weapon.attack}){cd}")
        
        hp_pct = self.player.hp / self.player.max_hp
        hp_bar = "❤️" * int(hp_pct * 10) + "🖤" * (10 - int(hp_pct * 10))
        print(f"HP: {hp_bar} ({self.player.hp}/{self.player.max_hp})")
        print(f"📍 Position: ({self.player.x}, {self.player.y})")
        
        inv = ", ".join(f"{i.emoji} {i.name}" for i in self.player.inv) if self.player.inv else "Empty"
        print(f"🎒 Inventory: {inv}")
        print(f"⏪ Undo: {len(self.undo_history)-1} available" if len(self.undo_history) > 1 else "⏪ Undo: Not available")
        print(f"\n{self.quests.display()}")
        
        # Debug info to help locate items
        if self.map.items:
            item_info = []
            for pos, item in self.map.items.items():
                item_info.append(f"{item.emoji} at ({pos[0]}, {pos[1]})")
            print(f"🔍 Items on map: {', '.join(item_info)}")
        
        if self.msg:
            print(f"\n📢 {self.msg}")
            self.msg = ""
        
        self.display_monsters()
    
    def handle_pickup(self):
        pos = (self.player.x, self.player.y)
        if pos in self.map.items:
            item = self.map.items[pos]
            self.player.inv.append(item)
            del self.map.items[pos]
            self.msg = f"📦 Picked up: {item.emoji} {item.name}"
    
    def check_victory(self):
        if all(m.hp <= 0 for m in self.monsters):
            self.quests.complete("Defeat All Monsters")
            self.msg = "🎉 Victory! All monsters defeated!"
            return True
        return False
    
    def run(self):
        self.msg = "🏴‍☠️ Welcome! Check your quest log and defeat all monsters!"
        
        while True:
            self.draw()
            
            if self.player.hp <= 0:
                print("\n💀 GAME OVER!")
                break
            if self.check_victory():
                print(f"\n🏆 VICTORY! Completed {self.map.diff} difficulty!")
                break
            
            try:
                cmd = input("\n> ").strip().lower()
            except KeyboardInterrupt:
                print("\n👋 Thanks for playing!")
                break
            
            if cmd == "quit":
                clear_output(wait=True)
                print("👋 Thanks for playing!")
                break
            
            elif cmd in {"north", "south", "east", "west", "n", "s", "e", "w"}:
                direction = {"n": "north", "s": "south", "e": "east", "w": "west"}.get(cmd, cmd)
                if self.player.move(direction, self.map, self.monsters):
                    self.save_simple_state(f"Moved {direction}")
                    self.handle_pickup()
                    self.move_monsters()
                    self.monster_attack()
                else:
                    self.msg = "❌ Can't move that way!"
            
            elif cmd.startswith("equip "):
                item_name = cmd[6:]
                for item in self.player.inv:
                    if item.name.lower() == item_name and item.attack > 0:
                        self.save_simple_state(f"Equipped {item.name}")
                        self.player.weapon = item
                        self.msg = f"⚔️ Equipped: {item.emoji} {item.name}"
                        break
                else:
                    self.msg = "❌ No such weapon!"
            
            elif cmd.startswith("use "):
                item_name = cmd[4:]
                for i, item in enumerate(self.player.inv):
                    if item.name.lower() == item_name and item.heal > 0:
                        self.save_simple_state(f"Used {item.name}")
                        result = item.use(self.player)
                        self.msg = f"💊 {result}"
                        del self.player.inv[i]
                        break
                else:
                    self.msg = "❌ No such item!"
            
            elif cmd == "attack":
                if self.player.can_attack():
                    monster = self.get_adjacent_monster()
                    if monster:
                        self.save_simple_state("Attacked monster")
                        self.player.attack()
                        weapon = self.player.weapon or self.fists
                        monster.hp = max(0, monster.hp - weapon.attack)
                        self.msg = f"⚔️ Hit {monster.name} for {weapon.attack}! " + (f"{monster.name} defeated!" if monster.hp == 0 else f"{monster.hp} HP left")
                    else:
                        self.msg = "❌ No monster nearby!"
                else:
                    self.msg = f"⏳ Cooling down! ({self.player.cooldown} turns)"
            
            elif cmd in {"undo", "u"}:
                self.undo()
            
            elif cmd in {"quests", "quest", "q"}:
                self.msg = self.quests.display()
            
            elif cmd in {"inventory", "inv", "i"}:
                self.msg = f"🎒 {', '.join(f'{i.emoji} {i.name}' for i in self.player.inv) if self.player.inv else 'Empty'}"
            
            elif cmd in {"help", "h", "?"}:
                self.msg = """🎮 Commands:
• north/south/east/west (n/s/e/w) - Move
• attack - Attack adjacent monster
• equip <item> - Equip weapon
• use <item> - Use item
• inventory/inv/i - Show inventory
• quests/q - Show quests
• undo/u - Undo last action
• help/? - This help
• quit - Exit"""
            
            else:
                self.msg = "❓ Unknown command. Type 'help' for commands."



In [12]:
# running game
game = Game(diff="easy")
game.run()

👋 Thanks for playing!
